## Assignment 4
#### Group 1: Tara Bode, Hankun Li

Part A: Text-based industry classification
Cluster the business sections extracted in assignment 3 into 50 clusters (using the cosine similarity) to group firms with similar business descriptions together.

Submit your code and the final output csv file, with CIK, conformed end of period, and the cluster number for each business section file.

Note: it is likely you will need to tokenize/word count each business section file before reading the next one. (If you try to read all business sections before doing any further processing you will probably run out of memory).

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities

In [2]:
import nltk
from nltk.corpus import stopwords
import string

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hli1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/hli1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.tokenize import word_tokenize

In [7]:
word_tokenize ("Joost is our professor") # need punkt to do this

['Joost', 'is', 'our', 'professor']

You need to simply take the.txt files from HiperGator and use the path (r'/blue/acg7849/share/BS/') 

so we can grab the length of each file for the csv file.

Also, we need to use the for loop to cluster the business section files and mimic the class code from yesterday to grab the top 20 common words.

In [8]:
#mydirectory = '/blue/acg7849/share/10Ks'
mydirectory = '/blue/acg7849/share/BS/item1'
file_list = glob.glob(mydirectory + '/*.txt')
# print(file_list) # success!

In [9]:
obj = pd.DataFrame(file_list)
obj.to_csv('file_list.csv', index = True) # csv preapared to use

# all functions

In [10]:
def getHeaderVariables():
    CIK = filing[ filing.find('CENTRAL INDEX KEY:') + 18  : filing.find('STANDARD') ]
    CIK = ''.join(CIK.split())
    #
    CCN = filing[ filing.find('COMPANY CONFORMED NAME:') + 23 : filing.find('CENTRAL INDEX KEY') ]
    CCN = CCN.rstrip().lstrip()
    #
    CST = filing[ filing.find('CONFORMED SUBMISSION TYPE:') + 26 : filing.find('PUBLIC DOCUMENT COUNT') ]
    CST = ''.join(CST.split())
    #
    CPR = filing[ filing.find('CONFORMED PERIOD OF REPORT:') + 27 : filing.find('FILED AS OF DATE') ]
    CPR = ''.join(CPR.split())
    #
    length = len(filing[ filing.find('<a id="ITEM1"></a>') : filing.find('<a id="ITEM1A"></a>') ])
    #

In [11]:
def cleanHTML(raw_html):
    cleanr = re.compile('<.*?>', flags=re.DOTALL)
    cleantext = re.sub(cleanr, '', raw_html)
    return replace_entities(cleantext)

In [12]:
def getBusinessSection():
    #
    BDS = filing[ filing.find('<a id="ITEM1"></a>') : filing.find('<a id="ITEM1A"></a>') ]
    #

# main loop

csv file, with CIK, conformed end of period, and file name (business section file) and the cluster number for each business section file.

In [16]:
badFiles_raw = []
for i in file_list[0:5]:
    print (i)
    # read the file
    with open(i) as f:
        filing = f.read()
        
    # grab the length of the file (word count)
    words = filing.split()
    print('Number of words in text file :', len(words))
    #print (words[0:10])
    # Ignore business section files with less than 1000 characters? (length).
    if len(words) < 5000: # test. should be 1000 here
        print ('bad')
        badFiles_raw.append (i)
    # should end this file and return to next one
    
    
    # tokenize
    #print (word_tokenize (filing[0:10])) # bit different from split - include punctuations



/blue/acg7849/share/BS/item1/267762.txt
Number of words in text file : 9719
/blue/acg7849/share/BS/item1/280603.txt
Number of words in text file : 8441
/blue/acg7849/share/BS/item1/267435.txt
Number of words in text file : 4208
bad
/blue/acg7849/share/BS/item1/275323.txt
Number of words in text file : 5852
/blue/acg7849/share/BS/item1/277539.txt
Number of words in text file : 5414


In [17]:
print(badFiles_raw)

['/blue/acg7849/share/BS/item1/267435.txt']


# 50 clusters (using the cosine similarity) to group firms with similar business descriptions together.

In [18]:
file_list = glob.glob(mydirectory + '/*.txt')

# list of files (raw text)
allFiles_raw = [] 

# process first 10 files
for file in file_list[0:10]:
    with open( i, encoding='utf-8') as f:
        content = f.read()
    # append file to allFiles
    allFiles_raw.append (content ) # plain text

In [28]:
# sklearn's Hierarchical clustering
# https://datascience.stackexchange.com/questions/22828/clustering-with-cosine-similarity
   
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
stopWords = set(stopwords.words('english') ) 
tfidf = TfidfVectorizer(stop_words='english')
# https://kavita-ganesan.com/how-to-use-countvectorizer/#.YLUpiKEpDUQ
X = CountVectorizer(allFiles_raw,stop_words=stopWords).fit_transform(allFiles_raw)
X = TfidfTransformer().fit_transform(X)
#Clustering
X = X.todense()
threshold = 0.5
# Z is the cosine distance matrix
Z = hierarchy.linkage(X,"average", metric="cosine")
# C are the clusters assigned
C = hierarchy.fcluster(Z, threshold, criterion="distance")

/home/hli1/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass input=['>Item\xa01.BusinessWe are an 140-year-old media enterprise with interests in local and national media brands. Founded in 1878, our motto is "Give light and the people will find their own way." Our mission is to do well by doing good — creating value for customers, employees and owners by informing, engaging and empowering those we serve. We serve audiences and businesses in our Local Media division through a portfolio of local television stations and their associated digital media products. Our Local Media division is one of the nation’s largest independent TV station ownership groups. Following the completion of the Raycom Media acquisition in January 2019 and the anticipated closing of the Cordillera Communications, LLC acquisition in the second quarter of 2019, we will have 51 television stations in 36 markets and a reach of more than one in five U.S. television households. We h

In [27]:



X = tfidf.fit_transform( allFiles_raw )
#Clustering
X = X.todense()
#num_clusters = 5
threshold = 0.5
# Z is the cosine distance matrix
Z = hierarchy.linkage(X,"average", metric="cosine")
# C are the clusters assigned
C = hierarchy.fcluster(Z, threshold, criterion="distance")

In [32]:
print(X[0,1])
print(X[0,2])

0.052999894000317994
0.0105999788000636


In [29]:
print(X)

[[0.00529999 0.05299989 0.01059998 ... 0.00529999 0.00529999 0.00529999]
 [0.00529999 0.05299989 0.01059998 ... 0.00529999 0.00529999 0.00529999]
 [0.00529999 0.05299989 0.01059998 ... 0.00529999 0.00529999 0.00529999]
 ...
 [0.00529999 0.05299989 0.01059998 ... 0.00529999 0.00529999 0.00529999]
 [0.00529999 0.05299989 0.01059998 ... 0.00529999 0.00529999 0.00529999]
 [0.00529999 0.05299989 0.01059998 ... 0.00529999 0.00529999 0.00529999]]


In [30]:
Z

array([[ 0.,  1.,  0.,  2.],
       [ 2., 10.,  0.,  3.],
       [ 3., 11.,  0.,  4.],
       [ 4., 12.,  0.,  5.],
       [ 5., 13.,  0.,  6.],
       [ 6., 14.,  0.,  7.],
       [ 7., 15.,  0.,  8.],
       [ 8., 16.,  0.,  9.],
       [ 9., 17.,  0., 10.]])

In [31]:
C

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [ ]:
# 20 most frequent words